In [ ]:
import os,shutil

In [ ]:
os.environ['KAGGLE_USERNAME']='rohanazad'
os.environ['KAGGLE_KEY']='7ab3fb30aaa1528df3ed28fd8a150646'

In [ ]:
!kaggle competitions download -c ashrae-energy-prediction

train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
weather_test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
weather_train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
building_metadata.csv: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import pandas as pd

In [ ]:
import zipfile

unzipped_train=zipfile.ZipFile('train.csv.zip')
unzipped_train.extractall()
unzipped_train.close()

unzipped_test=zipfile.ZipFile('test.csv.zip')
unzipped_test.extractall()
unzipped_test.close()

In [ ]:
# unzipped_weather_train=zipfile.ZipFile('weather_train.csv.zip')
# unzipped_weather_train.extractall()
# unzipped_weather_train.close()

# unzipped_weather_test=zipfile.ZipFile('weather_test.csv.zip')
# unzipped_weather_test.extractall()
# unzipped_weather_test.close()

In [ ]:
# weather_train=pd.read_csv('weather_train.csv')
# weather_test=pd.read_csv('weather_test.csv')
# building_met=pd.read_csv('building_metadata.csv')

In [ ]:
# total=weather_train.isnull().sum().sort_values(ascending=False)
# percent=(weather_train.isnull().sum()/weather_train.isnull().count()).sort_values(ascending=False)
# missing_values=pd.concat([total,percent],axis=1,keys=['total','percent'])
# missing_values.head(20)

In [ ]:
# total=building_met.isnull().sum().sort_values(ascending=False)
# percent=(building_met.isnull().sum()/building_met.isnull().count()).sort_values(ascending=False)
# missing_values=pd.concat([total,percent],axis=1,keys=['total','percent'])
# missing_values.head(20)

In [ ]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
weather_train_dir='/content/drive/My Drive/weather_train3.csv'
weather_train=pd.read_csv(weather_train_dir)
weather_test_dir='/content/drive/My Drive/weather_test2.csv'
weather_test=pd.read_csv(weather_test_dir)
building_met_dir='/content/drive/My Drive/building_metadata_save.csv'
building_met=pd.read_csv(building_met_dir)

In [ ]:
total=train.isnull().sum().sort_values(ascending=False)
percent=(train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_values=pd.concat([total,percent],axis=1,keys=['total','percent'])
missing_values.head(20)

,total,percent
meter_reading,0,0.0
timestamp,0,0.0
meter,0,0.0
building_id,0,0.0


In [ ]:
train.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


In [ ]:
print('shape of train:',train.shape)
print('shape of test:',test.shape)
print('shape of weather_train:',weather_train.shape)
print('shape of weather_test:',weather_test.shape)
print('shape of building_metadata:',building_met.shape)

shape of train: (20216100, 4)
shape of test: (41697600, 4)
shape of weather_train: (139788, 12)
shape of weather_test: (277243, 12)
shape of building_metadata: (1449, 6)


In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

**timestamp stores data as strings in a particular format. Year,month,day,time**

**Apply to_datetime to convert string to object**

In [ ]:
train['timestamp']=pd.to_datetime(train['timestamp'])
test['timestamp']=pd.to_datetime(test['timestamp'])
weather_train['timestamp']=pd.to_datetime(weather_train['timestamp'])
weather_test['timestamp']=pd.to_datetime(weather_test['timestamp'])

building_met['primary_use']=building_met['primary_use'].astype('category')

**Merging the three dataset to create train set**

In [ ]:
temp=train[['building_id']]
temp=temp.merge(building_met,on='building_id',how='left')
del temp['building_id']
train=pd.concat([train,temp],axis=1)



In [ ]:
train.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,4
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,4
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,4
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,4
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,4


In [ ]:
temp=train[['site_id','timestamp']]
temp=temp.merge(weather_train,on=['site_id','timestamp'],how='left')
del temp['site_id'],temp['timestamp']
train=pd.concat([train,temp],axis=1)

del temp,weather_train

In [ ]:
train.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,Month,Day_date,year
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,4,25.0,6.0,6.0,1.382557,1019.7,0.0,0.0,1.0,1.0,2016.0
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,4,25.0,6.0,6.0,1.382557,1019.7,0.0,0.0,1.0,1.0,2016.0
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,4,25.0,6.0,6.0,1.382557,1019.7,0.0,0.0,1.0,1.0,2016.0
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,4,25.0,6.0,6.0,1.382557,1019.7,0.0,0.0,1.0,1.0,2016.0
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,4,25.0,6.0,6.0,1.382557,1019.7,0.0,0.0,1.0,1.0,2016.0


In [ ]:
train.shape

(20216100, 19)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 19 columns):
 #   Column              Dtype         
---  ------              -----         
 0   building_id         int64         
 1   meter               int64         
 2   timestamp           datetime64[ns]
 3   meter_reading       float64       
 4   site_id             int64         
 5   primary_use         category      
 6   square_feet         int64         
 7   year_built          float64       
 8   floor_count         int64         
 9   air_temperature     float64       
 10  cloud_coverage      float64       
 11  dew_temperature     float64       
 12  precip_depth_1_hr   float64       
 13  sea_level_pressure  float64       
 14  wind_direction      float64       
 15  wind_speed          float64       
 16  Month               float64       
 17  Day_date            float64       
 18  year                float64       
dtypes: category(1), datetime64[ns](1), float

**Creating Hour column**

In [ ]:
import numpy as np

train['hour_datetime']=train['timestamp'].dt.hour.astype(np.int8)

del train['timestamp']

In [ ]:
train.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,Month,Day_date,year,hour_datetime
0,0,0,0.0,0,Education,7432,2008.0,4,25.0,6.0,6.0,1.382557,1019.7,0.0,0.0,1.0,1.0,2016.0,0
1,1,0,0.0,0,Education,2720,2004.0,4,25.0,6.0,6.0,1.382557,1019.7,0.0,0.0,1.0,1.0,2016.0,0
2,2,0,0.0,0,Education,5376,1991.0,4,25.0,6.0,6.0,1.382557,1019.7,0.0,0.0,1.0,1.0,2016.0,0
3,3,0,0.0,0,Education,23685,2002.0,4,25.0,6.0,6.0,1.382557,1019.7,0.0,0.0,1.0,1.0,2016.0,0
4,4,0,0.0,0,Education,116607,1975.0,4,25.0,6.0,6.0,1.382557,1019.7,0.0,0.0,1.0,1.0,2016.0,0


**Looking for missing values**

In [ ]:
total=train.isnull().sum().sort_values(ascending=False)
percent=(train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_values=pd.concat([total,percent],axis=1,keys=['total','percent'])
missing_values.head(20)

,total,percent
cloud_coverage,82890,0.0041
Day_date,82890,0.0041
Month,82890,0.0041
wind_speed,82890,0.0041
wind_direction,82890,0.0041
sea_level_pressure,82890,0.0041
precip_depth_1_hr,82890,0.0041
dew_temperature,82890,0.0041
year,82890,0.0041
air_temperature,82890,0.0041


In [ ]:
train=train.dropna()

In [ ]:
train.describe()

,building_id,meter,meter_reading,site_id,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,Month,Day_date,year,hour_datetime
count,2.013321e+07,2.013321e+07,2.013321e+07,2.013321e+07,2.013321e+07,2.013321e+07,2.013321e+07,2.013321e+07,2.013321e+07,2.013321e+07,2.013321e+07,2.013321e+07,2.013321e+07,2.013321e+07,2.013321e+07,2.013321e+07,20133210.0,2.013321e+07
mean,7.974179e+02,6.617699e-01,2.124405e+03,7.970224e+00,1.077780e+05,1.969521e+03,4.030442e+00,1.593926e+01,1.787219e+00,1.787219e+00,1.144656e+00,9.767713e+02,1.708236e+02,3.377249e+00,6.566733e+00,1.578433e+01,2016.0,1.150934e+01
std,4.264339e+02,9.310602e-01,1.535506e+05,5.092977e+00,1.171483e+05,2.014374e+01,1.665084e+00,1.098146e+01,2.003313e+00,2.003313e+00,6.979683e+00,1.961202e+02,1.107229e+02,2.261500e+00,3.441398e+00,8.802123e+00,0.0,6.923804e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.830000e+02,1.900000e+03,1.000000e+00,-2.890000e+01,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,2016.0,0.000000e+00
25%,3.910000e+02,0.000000e+00,1.830000e+01,3.000000e+00,3.252700e+04,1.964000e+03,4.000000e+00,8.300000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.011100e+03,8.000000e+01,2.100000e+00,4.000000e+00,8.000000e+00,2016.0,6.000000e+00
50%,8.940000e+02,0.000000e+00,7.875000e+01,9.000000e+00,7.270900e+04,1.975000e+03,4.000000e+00,1.670000e+01,1.109988e+00,1.109988e+00,0.000000e+00,1.015895e+03,1.700000e+02,3.100000e+00,7.000000e+00,1.600000e+01,2016.0,1.200000e+01
75%,1.176000e+03,1.000000e+00,2.679400e+02,1.300000e+01,1.391130e+05,1.975054e+03,4.000000e+00,2.390000e+01,2.873133e+00,2.873133e+00,0.000000e+00,1.020200e+03,2.700000e+02,4.600000e+00,1.000000e+01,2.300000e+01,2016.0,1.800000e+01
max,1.448000e+03,3.000000e+00,2.190470e+07,1.500000e+01,8.750000e+05,2.017000e+03,2.600000e+01,4.720000e+01,9.000000e+00,9.000000e+00,3.430000e+02,1.045500e+03,3.600000e+02,1.900000e+01,1.200000e+01,3.100000e+01,2016.0,2.300000e+01


In [ ]:
total=train.isnull().sum().sort_values(ascending=False)
percent=(train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_values=pd.concat([total,percent],axis=1,keys=['total','percent'])
missing_values.head(20)

,total,percent
hour_datetime,0,0.0
air_temperature,0,0.0
meter,0,0.0
meter_reading,0,0.0
site_id,0,0.0
primary_use,0,0.0
square_feet,0,0.0
year_built,0,0.0
floor_count,0,0.0
cloud_coverage,0,0.0


**Before applying pipeline transformation, chilled water dataset has been separated**

In [ ]:
dataset_chilled_water=train.loc[(train['meter']==1)]

In [ ]:
dataset_chilled_water.shape

(4161542, 19)

In [ ]:
# del_rows=dataset_chilled_water.loc[(dataset_chilled_water['site_id']==13)&(dataset_chilled_water['primary_use']=='Education')]
# del_rows

#dataset_chilled_water=dataset_chilled_water[(dataset_chilled_water.site_id!=13) & (dataset_chilled_water.primary_use!='Education')]
#dataset_chilled_water

In [ ]:
dataset_chilled_water.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,Month,Day_date,year,hour_datetime
172,161,1,0.0000,2,Education,11329,1954.388235,4,15.6,6.0,6.0,0.157738,1015.3,270.0,3.6,1.0,1.0,2016.0,0
174,162,1,0.0000,2,Education,98829,1968.000000,4,15.6,6.0,6.0,0.157738,1015.3,270.0,3.6,1.0,1.0,2016.0,0
176,163,1,4.5719,2,Education,72102,1970.000000,4,15.6,6.0,6.0,0.157738,1015.3,270.0,3.6,1.0,1.0,2016.0,0
181,166,1,209.8860,2,Lodging/residential,553210,2009.000000,4,15.6,6.0,6.0,0.157738,1015.3,270.0,3.6,1.0,1.0,2016.0,0
184,167,1,0.0000,2,Lodging/residential,86323,1956.000000,4,15.6,6.0,6.0,0.157738,1015.3,270.0,3.6,1.0,1.0,2016.0,0


In [ ]:
# target_chilled_water=dataset_chilled_water['meter_reading']

In [ ]:
# target_chilled_water.shape

**Feature engineering**

In [ ]:
def degToCompass(num):
  val=int((num/22.5))
  arr=[i for i in range(0,16)]
  return arr[(val%16)]

In [ ]:
beaufort=[(0,0,0.3),(1,0.3,1.6),(2,1.6,3.4),(3,3.4,5.5),(4,5.5,8),(5,8,10.8),
          (6,10.8,13.9),(7,13.9,17.2),(8,17.2,20.8),(9,20.8,24.5),(10,24.5,28.5),
          (11,28.5,33),(12,33,200)]

for item in beaufort:
  dataset_chilled_water.loc[(dataset_chilled_water['wind_speed']>=item[1])&(dataset_chilled_water['wind_speed']<item[2]),'beaufort_scale']=item[0]
dataset_chilled_water['wind_direction']=dataset_chilled_water['wind_direction'].apply(degToCompass)



/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [ ]:
def label_season(row):
  if row['air_temperature']<=25:
    return 'Winter'
  if row['air_temperature']>25:
    if row['air_temperature']<45:
      return 'Spring'
  if row['air_temperature']>=45:
    return 'Summer'


In [ ]:
dataset_chilled_water['season']=dataset_chilled_water.apply(lambda row:label_season(row),axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
dataset_chilled_water.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,Month,Day_date,year,hour_datetime,beaufort_scale,season
172,161,1,0.0000,2,Education,11329,1954.388235,4,15.6,6.0,6.0,0.157738,1015.3,12,3.6,1.0,1.0,2016.0,0,3.0,Winter
174,162,1,0.0000,2,Education,98829,1968.000000,4,15.6,6.0,6.0,0.157738,1015.3,12,3.6,1.0,1.0,2016.0,0,3.0,Winter
176,163,1,4.5719,2,Education,72102,1970.000000,4,15.6,6.0,6.0,0.157738,1015.3,12,3.6,1.0,1.0,2016.0,0,3.0,Winter
181,166,1,209.8860,2,Lodging/residential,553210,2009.000000,4,15.6,6.0,6.0,0.157738,1015.3,12,3.6,1.0,1.0,2016.0,0,3.0,Winter
184,167,1,0.0000,2,Lodging/residential,86323,1956.000000,4,15.6,6.0,6.0,0.157738,1015.3,12,3.6,1.0,1.0,2016.0,0,3.0,Winter


In [ ]:
correlation_matrix=dataset_chilled_water.corr()
correlation_matrix['meter_reading'].sort_values(ascending=False)

meter_reading         1.000000
square_feet           0.042789
year_built            0.030308
air_temperature       0.022007
Month                 0.013677
floor_count           0.008142
hour_datetime         0.006544
Day_date              0.005725
sea_level_pressure    0.004839
precip_depth_1_hr    -0.000987
dew_temperature      -0.004249
cloud_coverage       -0.004249
beaufort_scale       -0.005193
wind_direction       -0.005224
wind_speed           -0.005314
building_id          -0.011695
site_id              -0.016233
meter                      NaN
year                       NaN
Name: meter_reading, dtype: float64

In [ ]:
drop_cols=['meter_reading','precip_depth_1_hr','dew_temperature','cloud_coverage']

In [ ]:
training_chilled_water=dataset_chilled_water.drop(drop_cols,axis=1)

In [ ]:
training_chilled_water.head()

,building_id,meter,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,sea_level_pressure,wind_direction,wind_speed,Month,Day_date,year,hour_datetime,beaufort_scale,season
172,161,1,2,Education,11329,1954.388235,4,15.6,1015.3,12,3.6,1.0,1.0,2016.0,0,3.0,Winter
174,162,1,2,Education,98829,1968.000000,4,15.6,1015.3,12,3.6,1.0,1.0,2016.0,0,3.0,Winter
176,163,1,2,Education,72102,1970.000000,4,15.6,1015.3,12,3.6,1.0,1.0,2016.0,0,3.0,Winter
181,166,1,2,Lodging/residential,553210,2009.000000,4,15.6,1015.3,12,3.6,1.0,1.0,2016.0,0,3.0,Winter
184,167,1,2,Lodging/residential,86323,1956.000000,4,15.6,1015.3,12,3.6,1.0,1.0,2016.0,0,3.0,Winter


In [ ]:
target_chilled_water=dataset_chilled_water['meter_reading']

In [ ]:
target_chilled_water.shape

(4161542,)

**Apply pipeline transformation to training_chilled_water**

In [ ]:
num_cols=['square_feet','floor_count','air_temperature',
                 'sea_level_pressure','wind_direction','wind_speed']
cat_cols=['primary_use','season']

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_attribs=list(num_cols)
cat_attribs=['primary_use']

num_pipeline=Pipeline([
    ('std_scaler',StandardScaler())
])

full_pipeline=ColumnTransformer([
    ('num',num_pipeline,num_attribs),
    ('cat',OneHotEncoder(),cat_attribs)
])

training_prepared=full_pipeline.fit_transform(training_chilled_water)
#('num',num_pipeline,num_attribs),

In [ ]:
x_train=training_prepared[:1000000]
x_val=training_prepared[1000000:]
y_train=target_chilled_water[:1000000]
y_val=target_chilled_water[1000000:]

In [ ]:
print('shape of x_train:',x_train.shape)
print('shape of x_val:',x_val.shape)
print('shape of y_train:',y_train.shape)
print('shape of y_val:',y_val.shape)

shape of x_train: (1000000, 20)
shape of x_val: (3161542, 20)
shape of y_train: (1000000,)
shape of y_val: (3161542,)


**Working with test set**

In [ ]:
#test.head()

In [ ]:
# test.shape

In [ ]:
# import pandas as pd

# temp=test[['building_id']]
# temp=temp.merge(building_met,on='building_id',how='left')
# del temp['building_id']
# test=pd.concat([test,temp],axis=1)

In [ ]:
# test.head()

In [ ]:
# temp=test[['site_id','timestamp']]
# temp=temp.merge(weather_test,on=['site_id','timestamp'],how='left')
# del temp['timestamp'],temp['site_id']
# test=pd.concat([test,temp],axis=1)

# del temp,weather_test

In [ ]:
# test.head()

In [ ]:
# total=test.isnull().sum().sort_values(ascending=False)
# percent=(test.isnull().sum()/test.isnull().count()).sort_values(ascending=False)
# missing_values=pd.concat([total,percent],axis=1,keys=['total','percent'])
# missing_values.head(20)

In [ ]:
# test=test.dropna()

In [ ]:
# test.describe()

In [ ]:
# test_chilled_water= test.loc[(test['meter']==1)]

In [ ]:
# test_chilled_water.shape

In [ ]:
# test_drop_cols=['row_id']

In [ ]:
# testing=test_chilled_water.drop(test_drop_cols,axis=1)

In [ ]:
# testing.shape

**Applying feature engineering in test set**

In [ ]:
# for item in beaufort:
#   testing.loc[(testing['wind_speed']>=item[1])&(testing['wind_speed']<item[2]),beaufort_scale]=item[0] 
# testing['wind_direction']=testing['wind_direction'].apply(degToCompass)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt

lin_reg=LinearRegression()
lin_reg.fit(x_train,y_train)
lin_predict=lin_reg.predict(x_val)
lin_mse=mean_squared_error(lin_predict,y_val)
lin_rmse=sqrt(lin_mse)
lin_rmse

9179.982658552519

Linear regression RMSE 2559.0594065652162

**Applying 2 degree polynomial**

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly_features=PolynomialFeatures(degree=2,include_bias=False)
x_poly=poly_features.fit_transform(x_train[:1000000])
lin_reg=LinearRegression()
lin_reg.fit(x_poly,y_train[:1000000])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
x_val_poly=poly_features.transform(x_val)
predict=lin_reg.predict(x_val_poly)
degree2_mse=mean_squared_error(predict,y_val)
degree2_rmse=sqrt(degree2_mse)
degree2_rmse

17317.280549717925

2nd degree polynomial RMSE 2486.5616840162443

**Applying 3 degree polynomial. 3 or higher degree polynomial will have to be done through virtual server**

In [ ]:
# from sklearn.preprocessing import PolynomialFeatures

# poly3_features=PolynomialFeatures(degree=3,include_bias=False)
# x_poly=poly3_features.fit_transform(x_train[:1000000])
# lin_reg=LinearRegression()
# lin_reg.fit(x_poly,y_train[:1000000])

In [ ]:
# x_val_poly=poly3_features.transform(x_val)
# predict=lin_reg.predict(x_val_poly)
# degree3_mse=mean_squared_error(predict,y_val)
# degree3rmse=sqrt(degree2_mse)
# degree3_rmse

**Using grid search with Ridge regression**

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
param_grid=[
            {'alpha':[0.001,0.003,0.01,0.03,0.1,0.3,1,3]}
]

ridge_reg=Ridge()

grid_search=GridSearchCV(ridge_reg,param_grid,cv=5,
                         scoring='neg_mean_squared_error',
                         return_train_score=True)

grid_search.fit(x_train[:1000000],y_train[:1000000])

GridSearchCV(cv=5, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid=[{'alpha': [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='neg_mean_squared_error', verbose=0)

In [ ]:
grid_search.best_params_

{'alpha': 3}

Ridge regression alpha=0.3

**Ridge regression with alpha=0.3**

In [ ]:
from sklearn.metrics import mean_squared_error

ridge_reg=Ridge(alpha=3,solver='cholesky')
ridge_reg.fit(x_train[:1000000],y_train[:1000000])
ridge_pred=ridge_reg.predict(x_val)
ridge_mse=mean_squared_error(ridge_pred,y_val)
ridge_rmse=np.sqrt(ridge_mse)
ridge_rmse

17321.296525407557

Ridge regression RMSE 2496.5487098674917

**Using grid search with Ridge regression**

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

param_grid=[
            {'alpha':[0.001,0.003,0.01,0.03,0.1,0.3,1,3]}
]

lasso_reg=Lasso()

grid_search=GridSearchCV(lasso_reg,param_grid,cv=5,
                         scoring='neg_mean_squared_error',
                         return_train_score=True)

grid_search.fit(x_train[:1000000],y_train[:1000000])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14308499512.14206, tolerance: 49440643.923286386
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4574734769.701843, tolerance: 46560643.66012349
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13516799288.64444, tolerance: 44540331.31990401
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11811882522

KeyboardInterrupt: ignored

In [ ]:
grid_search.best_params_

Lasso regression alpha= 0.001

**Lasso regression with alpha= 0.001**

In [ ]:
from sklearn.linear_model import Lasso

lasso_reg=Lasso(alpha=0.001)
lasso_reg.fit(x_train[:1000000],y_train[:1000000])
lasso_pred=lasso_reg.predict(x_val)
lasso_mse=mean_squared_error(lasso_pred,y_val)
lasso_rmse=np.sqrt(lasso_mse)
lasso_rmse

Lasso regression RMSE 2496.5489890912604

**Grid search with elastic net**

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet

param_grid=[
            {'alpha':[0.001,0.003,0.01,0.03,0.1,0.3,1,3]}
]

elastic_reg=ElasticNet()

grid_search=GridSearchCV(elastic_reg,param_grid,cv=5,
                         scoring='neg_mean_squared_error',
                         return_train_score=True)

grid_search.fit(x_train[:1000000],y_train[:1000000])

In [ ]:
grid_search.best_params_

Elastic net alpha=0.001

**Applying elastic net with alpha=0.001**

In [ ]:
from sklearn.linear_model import ElasticNet

elastic_reg=ElasticNet(alpha=0.001)
elastic_reg.fit(x_train[:1000000],y_train[:1000000])
elastic_pred=elastic_reg.predict(x_val)
elastic_mse=mean_squared_error(elastic_pred,y_val)
elastic_rmse=np.sqrt(elastic_mse)
elastic_rmse

Elastic net RMSE= 2496.5731476766564

**Grid search to get the optimal learning rate**

In [ ]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.linear_model import SGDRegressor

# param_grid=[
#             {'eta0':[0.001,0.003,0.01,0.03,0.1,0.3,1,3]}
# ]

# sgd_reg=SGDRegressor(max_iter=1000,tol=1e-3,penalty='l2')

# grid_search=GridSearchCV(sgd_reg,param_grid,cv=5,
#                          scoring='neg_mean_squared_error',
#                          return_train_score=True)
# grid_search.fit(x_train[:1000000],y_train[:1000000])

In [ ]:
# grid_search.best_params_

**Stochastic gradient descent**

In [ ]:
# from sklearn.linear_model import SGDRegressor
# from sklearn. metrics import mean_squared_error

# sgd_reg=SGDRegressor(max_iter=1000,tol=1e-3,penalty=None,eta0=0.001)
# sgd_reg.fit(x_train[:1000000],y_train[:1000000])
# sgd_pred=sgd_reg.predict(x_val)
# sgd_mse=mean_squared_error(sgd_pred,y_val)
# sgd_rmse=np.sqrt(sgd_mse)
# sgd_rmse

Stochastic gradient descent RMSE with eta0=0.001 is 2496.5836952519935

**Decision tree**

In [ ]:
# from sklearn.tree import DecisionTreeRegressor

# tree_reg=DecisionTreeRegressor()
# tree_reg.fit(x_train[:1000000],y_train[:1000000])
# tree_pred=tree_reg.predict(x_val)
# tree_mse=mean_squared_error(tree_pred,y_val)
# tree_rmse=np.sqrt(tree_mse)
# tree_rmse

Decision tree RMSE = 2478.8828020681417

**Ensemble**

In [ ]:
from sklearn.ensemble import VotingRegressor
from sklearn.svm import LinearSVR
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor

svm_reg= LinearSVR()


voting_reg=VotingRegressor(
    estimators=[('lr',lin_reg),('ridge',Ridge(alpha=3)),('lasso',Lasso(alpha=0.001)),
                ('elastic',ElasticNet(alpha=0.001)),('sgd_reg',SGDRegressor(max_iter=1000,tol=1e-3,penalty=None,eta0=0.001)),
                ('tree',DecisionTreeRegressor())]
)

voting_reg.fit(x_train[:1000000],y_train[:1000000])

VotingRegressor(estimators=[('lr',
                             LinearRegression(copy_X=True, fit_intercept=True,
                                              n_jobs=None, normalize=False)),
                            ('ridge',
                             Ridge(alpha=3, copy_X=True, fit_intercept=True,
                                   max_iter=None, normalize=False,
                                   random_state=None, solver='auto',
                                   tol=0.001)),
                            ('lasso',
                             Lasso(alpha=0.001, copy_X=True, fit_intercept=True,
                                   max_iter=1000, normalize=False,
                                   positive=False, precomp...
                                          validation_fraction=0.1, verbose=0,
                                          warm_start=False)),
                            ('tree',
                             DecisionTreeRegressor(ccp_alpha=0.0,
                  

In [ ]:
voting_reg_pred=voting_reg.predict(x_val)
voting_reg_mse=mean_squared_error(voting_reg_pred,y_val)
voting_reg_rmse=np.sqrt(voting_reg_mse)
voting_reg_rmse

9172.552288267258

Voting regressor RMSE 2481.9209460610778

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import GridSearchCV

# param_grid=[
#             {'n_estimators':[3,10,30,100]}
# ]

# forest_reg=RandomForestRegressor()

# grid_search=GridSearchCV(forest_reg,param_grid,cv=5,
#                          scoring='neg_mean_squared_error',
#                          return_train_score=True)

# grid_search.fit(x_train[:90000],y_train[:90000])

In [ ]:
# grid_search.best_estimator_

In [ ]:
# from sklearn. ensemble import RandomForestRegressor

# forest_reg=RandomForestRegressor(n_estimators=150,max_features=9,n_jobs=-1)
# forest_reg.fit(x_train[:1000000],y_train[:1000000])

In [ ]:
# pred=forest_reg.predict(x_val)

In [ ]:
# mse=mean_squared_error(pred,y_val)
# mse

In [ ]:
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import train_test_split

# def plot_learning_curves(model, X, y):
#     chilled_x_train=training_prepared[:10000]
#     chilled_x_val=training_prepared[10000:20000]
#     chilled_y_train=target_chilled_water[:10000]
#     chilled_y_val=target_chilled_water[10000:20000]
#     train_errors, val_errors = [], []
#     for m in range(1, len(chilled_x_train)):
#         model.fit(chilled_x_train[:m], chilled_y_train[:m])
#         y_train_predict = model.predict(chilled_x_train[:m])
#         y_val_predict = model.predict(chilled_x_val)
#         y_train_mse=mean_squared_error(chilled_y_train[:m], y_train_predict)
#         y_val_mse=mean_squared_error(chilled_y_val, y_val_predict)
#         y_train_rmse=np.sqrt(y_train_mse)
#         y_val_rmse=np.sqrt(y_val_mse)
#         train_errors.append(y_train_rmse)
#         val_errors.append(y_val_rmse)
#         for index in enumerate(train_errors):
#           print(index)
        

#     plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
#     plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")   
#     plt.xlabel("Training set size", fontsize=14) 
#     plt.ylabel("RMSE", fontsize=14)       

In [ ]:
# from sklearn.linear_model import LinearRegression

# lin_reg = LinearRegression()
# plot_learning_curves(lin_reg, training_prepared, target_chilled_water)                           
# plt.show()